In [8]:
# libraries
import os
import csv
import time
from NYT_parser import NYTArticle
from utilities import *

In [9]:
# get pathways
train_paths = data_setup.get_paths('train')
article = NYTArticle.from_file(os.path.join('data', train_paths[0][0]))

In [10]:
# set destination folder and make folders if needed
dest_folder = 'transdata/train/'
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

In [6]:
# attempt a single file output
print("Writing", dest_folder+article.docid)
article.simple_csv_output(dest_folder)

Writing transdata/train/739212


In [7]:
# attempt 50 article write minus filtered out
for path in train_paths[:50]:
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        print("Writing", dest_folder+article.docid)
        article.simple_csv_output(dest_folder)

Writing transdata/train/1397928
Writing transdata/train/1609775
Writing transdata/train/297100
Writing transdata/train/448007
Writing transdata/train/1463610
Writing transdata/train/590096
Writing transdata/train/762936
Writing transdata/train/1570877
Writing transdata/train/1725268
Writing transdata/train/1647864
Writing transdata/train/1251662
Writing transdata/train/551924
Writing transdata/train/74723
Writing transdata/train/1535917
Writing transdata/train/744457
Writing transdata/train/97129
Writing transdata/train/899133
Writing transdata/train/791668
Writing transdata/train/779448
Writing transdata/train/1287650
Writing transdata/train/927546
Writing transdata/train/1587613
Writing transdata/train/638654
Writing transdata/train/870914
Writing transdata/train/800527
Writing transdata/train/1584905
Writing transdata/train/1287373
Writing transdata/train/1337862
Writing transdata/train/1646102
Writing transdata/train/1363692
Writing transdata/train/50986
Writing transdata/train/148

In [11]:
# write all filtered articles in train set
opened_article = 0
written_article = 0
start = time.time()
for path in train_paths:
    opened_article += 1
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        written_article += 1
        article.simple_csv_output(dest_folder)
    if opened_article % 10000 == 0:
        print("Opened", opened_article,"articles. Output", written_article,"articles.")
print(f"\n... processed {opened_article} files in {time.time() - start} seconds")


... processed 1001 files in 2.489995241165161 seconds
